In [ ]:
%load_ext blackcellmagic

#### Imports and setup

In [ ]:
import os
import numpy as np
import pandas as pd
import deepdish as dd
import h5py
import pickle
import nibabel.freesurfer.mghformat as mgh
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from spacestream.core.constants import SUBJECTS, CORE_ROI_NAMES
from spacestream.core.paths import DATA_PATH, RESULTS_PATH
from spacestream.utils.general_utils import sem
from spacestream.utils.get_utils import get_mapping

In [ ]:
seeds = [0,1,2,3,4]
hemis = ["lh","rh"]
sw_list = [
            "0.0",
            "0.1",
            "0.25",
            "0.5",
            "1.25",
            "2.5",
            "25.0",
        ]
sup_case = ["simCLR", "supervised"]
colors = {}
colors["simCLR"], colors["supervised"] = "#720298", "#B59410"

#### Load and format data

##### Load effective dimensionality calculations

In [ ]:
load_dir = os.path.join(
    RESULTS_PATH,
    "analyses/effective_dim/",
)

In [ ]:
#brain
brain_ED = {}
for hemi in hemis:
    for sidx, subj in enumerate(SUBJECTS):
        context = f"subjects_by_stream_{hemi}.pkl"
        fname = os.path.join(load_dir, context)
        brain_ED[hemi] = pickle.load(open(fname, "rb"))
total_subject_ED = np.vstack((brain_ED['lh'],brain_ED['rh']))

In [ ]:
#model
ED = np.zeros((2,2,7,5,8,3))
for hidx, hemi in enumerate(hemis):
    for supidx, sup in enumerate(sup_case):
        supervised_stem = "supervised_" if supidx==1 else ""

        context = f"{supervised_stem}ED_by_stream_{hemi}.pkl"
        fname = os.path.join(load_dir, context)
        ED[hidx,supidx] = pickle.load(open(fname, "rb"))

##### Load functional correlations and organize data

In [ ]:
def correct_for_voxel_noise_ceiling(NC, mapping):

    brain_r = np.sqrt(
        NC[mapping["winning_idx"].astype(int)] / 100
    )  # convert from R^2 to r
    mapping["winning_roi"] = mapping["winning_roi"].astype(np.float32)

    if np.sum(np.isinf(mapping["winning_test_corr"])) > 0:
        mapping["winning_test_corr"][np.isinf(info["winning_test_corr"])] = np.nan

    corrected = mapping["winning_test_corr"] / brain_r
    corrected[mapping["winning_test_corr"] == 0] = np.nan

    return corrected

In [ ]:
full_results = {
    'Spatial Weight': [],
    'Type': [],
    "Combined": [],
    'Seed': [],
    'Stream': [],
    'Subject': [],
    'Hemi': [],
    'ED': [],
    'Corr': [],
}

for hidx, hemi in enumerate(hemis):
    
    for sidx, subj in enumerate(SUBJECTS):

        # get ROI info
        mgh_file = mgh.load(DATA_PATH + "brains/" + hemi + ".ministreams.mgz")
        streams = mgh_file.get_fdata()[:, 0, 0].astype(int)
        # get noise ceiling estimates
        mgh_file = mgh.load(
            DATA_PATH + "brains/NC/subj" + subj + "/" + hemi + ".nc_3trials.mgh"
        )
        NC = mgh_file.get_fdata()[:, 0, 0]
        NC_trim = NC[streams != 0]
        NC_trim[NC_trim == 0] = np.nan  # Set all 0s to nans to avoid dividing by 0


        for supidx, sup in enumerate(sup_case):
            for widx, weight in enumerate(sw_list):

                for midx, seed in enumerate(seeds):
                    mapping = get_mapping(
                        subj_name="subj" + str(subj),
                        spatial_weight=weight,
                        model_seed=seed,
                        supervised=supidx,
                        hemi=hemi,
                    )
                    corrected = correct_for_voxel_noise_ceiling(NC_trim, mapping)


                    for ridx, r in enumerate(CORE_ROI_NAMES):
                        
                        full_results["Spatial Weight"].append(str(weight))
                        full_results["Type"].append(sup)
                        full_results["Combined"].append(str(weight)+"_"+str(supidx))
                        full_results["Seed"].append(seed)
                        full_results["Subject"].append(subj)
                        full_results["Stream"].append(r)
                        full_results["ED"].append(ED[hidx,supidx,widx, midx, sidx, ridx])
                        full_results["Hemi"].append(hemi)
                        full_results["Corr"].append(
                            np.nanmean(corrected[mapping["winning_roi"] == ridx + 5])
                        )


In [ ]:
full_df = pd.DataFrame(full_results)
df_avg = pd.DataFrame(full_df.groupby(["Spatial Weight", "Type","Combined", "Subject", "Stream", "Seed"])["ED","Corr"].mean()).reset_index()

In [ ]:
df_avg

In [ ]:
df_avg.to_csv('/share/kalanit/users/dfinzi/Desktop/processed_spacestream_csvs/F03_C.csv', index=False)  

In [ ]:
## load subject2subject estimates
s2s_corrected_by_stream= np.zeros((len(SUBJECTS),len(seeds),len(CORE_ROI_NAMES), len(hemis)))

for hidx, hemi in enumerate(hemis):
    
    for sidx, subj in enumerate(SUBJECTS):

        for seedix, seed in enumerate(seeds):

            load_path = (RESULTS_PATH
                            + "mappings/one_to_one/voxel2voxel/target_subj"
                            + subj
                            + "/mode_"
                            + hemi
                            + "_ministreams_HVA_only_radius5_max_iters100_constant_radius_2.0dist_cutoff_constant_dist_cutoff_spherical"
                            + ("_CV_seed" + str(seed))
                            + "_final_voxel2voxel_correlation_info.hdf5"
                        )
            with h5py.File(load_path, "r") as f:

                for r, ridx in enumerate(CORE_ROI_NAMES):
                    s2s_corrected_by_stream[sidx,seedix,r,hidx] =  np.nanmean(f['corrected_test_corr'][:][f['winning_roi'][:] == r+5])
across_seed_corrected_mean = np.mean(np.mean(s2s_corrected_by_stream,axis=-1),axis=1)

In [ ]:
across_seed_corrected_mean


In [ ]:
np.mean(total_subject_ED.reshape(8,2,3),axis=1)

#### Plotting

In [ ]:
for ridx, roi in enumerate(CORE_ROI_NAMES):
    fig, ax = plt.subplots(figsize=(7, 9))
    sns.set_theme(style="ticks")

    kwargs = {
        "edgecolor": "w",  # for edge color
        "linewidth": 0.01,  # line width of spot
    }
    df_roi = df_avg[df_avg["Stream"].isin([roi])]

    xmin = 0
    xmax = 1

    vals = across_seed_corrected_mean[:, ridx]
    y1 = np.mean(vals) - np.std(vals)
    y2 = np.mean(vals) + np.std(vals)
    ax.axhspan(y1, y2, xmin=xmin, xmax=xmax, color="lightgray", alpha=0.75)
    y1 = (
        np.mean(total_subject_ED, axis=0)[ridx]
        + np.std(total_subject_ED, axis=0)[ridx]
    )
    y2 = (
        np.mean(total_subject_ED, axis=0)[ridx]
        - np.std(total_subject_ED, axis=0)[ridx]
    )
    ax.axvspan(y1, y2, color="lightgray", alpha=0.75)  # color=color_mappings[roi],

    sns.scatterplot(
        data=df_roi[df_roi["Type"].isin(["supervised"])][
            ~df_roi[df_roi["Type"].isin(["supervised"])]["Spatial Weight"].isin(["2.5"])
        ],
        x="ED",
        y="Corr",
        hue="Combined",
        size="Spatial Weight",
        sizes=[15, 25, 50, 75, 100, 250],  # [15, 25, 50, 75, 100, 250, 150],
        alpha=0.3,
        palette=["#FFFFFF"],
        edgecolor=[
            "#B59410"
        ],  # [color_mappings[roi]], #["#4d7fff","#006600","#DC267F"],
        facecolor="w",
        linewidth=2,
        zorder=10,
        ax=ax,
    )
    sns.scatterplot(
        data=df_roi[df_roi["Type"].isin(["simCLR"])][
            ~df_roi[df_roi["Type"].isin(["simCLR"])]["Spatial Weight"].isin(
                ["0.25", "0.5"]
            )
        ],
        x="ED",
        y="Corr",
        hue="Combined",
        size="Spatial Weight",
        sizes=[15, 25, 100, 150, 250],  # [15, 25, 75, 100, 150, 250, 50],
        palette=["#FFFFFF"],
        edgecolor=["#720298"],
        facecolor="w",
        linewidth=2,
        alpha=0.3,
        zorder=10,
        ax=ax,
    )

    sns.scatterplot(
        data=df_roi[df_roi["Type"].isin(["supervised"])][
            df_roi[df_roi["Type"].isin(["supervised"])]["Spatial Weight"].isin(["2.5"])
        ],
        x="ED",
        y="Corr",
        hue="Combined",
        size="Spatial Weight",
        sizes=[150],  # [15, 25, 50, 75, 100, 250, 150],
        palette=["#B59410"],
        zorder=10,
        ax=ax,
    )
    sns.scatterplot(
        data=df_roi[df_roi["Type"].isin(["simCLR"])][
            df_roi[df_roi["Type"].isin(["simCLR"])]["Spatial Weight"].isin(
                ["0.25", "0.5"]
            )
        ],
        x="ED",
        y="Corr",
        hue="Combined",
        size="Spatial Weight",
        sizes=[50, 75],  # [15, 25, 75, 100, 150, 250, 50],
        palette=["#720298"],  # [color_mappings[roi]], #["#4d7fff","#006600","#DC267F"],
        zorder=10,
        ax=ax,
    )

    ax.set_xscale("log")
    ax.set_ylim(0, 0.43)

    plt.legend([], [], frameon=False)
    ax.set_xlabel("Effective Dimensionality")
    ax.set_ylabel("Correlation")

    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)

    save_path = RESULTS_PATH + "figures/3C_" + roi + "_SD.png"
    fig.savefig(save_path, dpi=100)